In [99]:
import numpy as np
import sys as sys
import random as rd
import tensorflow as tf
import gzip
from __future__ import division
import pandas as pd
import os
import math
from __future__ import absolute_import
from __future__ import print_function
import time

In [100]:
n_opt_epoch = 2
model = 'lstm'
layers = 1
n_samples = 10
n_users = 1000
te_users = int(n_users * 0.2)
n_feature = 300
n_user_feature = 310
lr_rat = 0.001
num_video = 1000
beta = 0.01
batch_size = 128
n_hidden = 32

In [101]:
def block_tri_mat(length, batch_size):
    upper_triangular_ones = np.float32(np.triu(np.ones((length, length))))
    repeated_tri = np.float32(np.kron(np.eye(batch_size), upper_triangular_ones))
    return repeated_tri

In [102]:
def negtv_sample(to_sample, pop, batch_size, length_max, num_to_sample):
    num_rw_neg_mtrx = length_max * batch_size
    s_neg = (num_rw_neg_mtrx, num_to_sample)
    neg_sample = np.zeros(s_neg)
    start = 0
    for i in range(batch_size):
        not_watch = (np.array(to_sample.iloc[i,1])).reshape(-1)
        pop_not_watch_norm = pop[not_watch]/sum(pop[not_watch])
        if (i > 0):
            end = length_max + end
        else:
            end = length_max
        neg_sample[start:end] = np.random.choice(not_watch ,(length_max ,num_to_sample)
                                               ,p = pop_not_watch_norm ,replace = False)
        start = np.copy(end)
    # get for each row the column to pick      
    rep_seq = np.repeat(range(neg_sample.shape[0]) ,num_to_sample).reshape(-1,1) 
    idx_p_w = np.concatenate((rep_seq ,neg_sample.reshape(-1,1)) ,axis = 1)
    
    return(neg_sample ,idx_p_w ,rep_seq)

In [103]:
def np_pad_tr_x(x_tr, batch_size, str_idx, zero_array_x, length_vec):
    start = 0
    for i in range(batch_size):
        if i > 0:
            end = end + length_vec[i] 
        else:
            end = length_vec[0]
        zero_array_x[ str_idx[i] : (str_idx[i] + length_vec[i]) ] = x_tr[ start : end ] 
        start = end
    return(zero_array_x)


def np_pad_tr_x_n_tr_y(x_tr, y_tr, batch_size, str_idx, zero_array_x, zero_array_y, length_vec):
    start = 0
    for i in range(batch_size):
        if i > 0:
            end = end + length_vec[i] 
        else:
            end = length_vec[0]
        zero_array_x[ str_idx[i] : (str_idx[i] + length_vec[i]) ] = x_tr[ start : end ] 
        zero_array_y[ str_idx[i] : (str_idx[i] + length_vec[i]) ] = y_tr.iloc[ start : end, (y_tr.shape[1] - 1) ] 
        start = end

    return(zero_array_x, zero_array_y)


def range_bet_col_t_col_n_append(col_1 ,col_2):
    app_ranges=[]
    for i in range(col_1.shape[0]):
        single_range = range((col_1[i]).astype(int) ,(col_2[i]).astype(int))
        app_ranges = np.append(app_ranges,single_range)
    return(app_ranges)


def algeb_geom_series(mode ,start ,jump ,length):
    u = np.empty((length,))
    u[0] = start
    u[1:] = jump
    if (mode == 0):
        series=np.cumsum(u)
    if (mode == 1):
        series=np.cumprod(u)
    return(series)

def sort_pd_df_by_ext_vec(df,ext_sor_vec,cols):
    df_s = df[((df[cols[0]]).astype(int)).isin(ext_sor_vec)] #
    df_s['sort_cat'] = pd.Categorical(df_s[cols[0]],categories = ext_sor_vec,ordered = True)
    if len(cols) > 1:
        df_s.sort_values(['sort_cat',cols[1]] ,inplace = True)
    
    else:
        df_s.sort_values(['sort_cat'],inplace = True) 
    
    df_s.reset_index(inplace = True)
    df_ = df_s.drop(['sort_cat','index'] ,axis = 1)
    
    return(df_)

def trainSamples(viewers,videos,probab,viewerFeat,videoFeat,contxFeat):
    trData = {} # trData = Dictionary with training data. This is histrory of viewer and video iteraction
    X = {} # X  = Dictionary with viewer features as arrays
    Y ={} # Y  = Dictionary with video features as arrays
    for i in range(viewers):
        X[i] = np.random.rand(viewerFeat)
        a = 0 # timing of the video for a particular user,... 
                #to give the order in which the videos have been watched
        for j in range(videos):
            if int(np.random.binomial(1,probab ,1)[0]):
                trData[(i,j,a)] = np.random.rand(contxFeat)
                a+=1 # when a video is watched, we increase the value of a by 1 
            if i==0:
                Y[j] = np.random.rand(videoFeat)
    return X,Y,trData


def attach_zeros_to_np_arr(array ,size_to_att ,axis):
    size = size_to_att
    if(axis == 0):
        new_arr = np.append(array ,np.zeros(size) ,axis = 0)
    if(axis == 1):
        new_arr = np.concatenate((array ,np.zeros(size)) ,axis = 1)
    return(new_arr)


def rnn_model(model ,n_hidden ,layers):
    if (layers == 1):
        if(model == 'lstm'):
            try:
                cell = tf.nn.rnn_cell.LSTMCell(n_hidden ,state_is_tuple=True,reuse=tf.get_variable_scope().reuse)
            except ValueError:
                print('yo')
                cell = tf.nn.rnn_cell.LSTMCell(n_hidden ,state_is_tuple=True, reuse=True)
        else:
            cell = tf.nn.rnn_cell.GRUCell(n_hidden)   
    else:
        if(model == 'lstm'):
            lstm = tf.nn.rnn_cell.LSTMCell(n_hidden ,state_is_tuple=True)
            cell = tf.nn.rnn_cell.MultiRNNCell([lstm]*layers)

        else:
            gru = tf.nn.rnn_cell.GRUCell(n_hidden)
            cell = tf.nn.rnn_cell.MultiRNNCell([gru] * layers)
    return(cell)

viewers = 1000  #number of viewers
videos =1000  #number of videos
probab = 0.05  #probability of a viewer watching any one video
viewerFeat = 310  #number of features describing a veiwer
videoFeat = 300   #number of features describing a video
contxFeat = 15 # number of contextual features
# X  = Dictionary with viewer features as arrays
# Y  = Dictionary with video features as arrays
# trData = Dictionary with training data. This is histrory of viewer and video iteraction
X ,Y ,trData = trainSamples(viewers ,videos ,probab ,viewerFeat ,videoFeat ,contxFeat)

user_feat_inp = np.array([X[key] for key in sorted(X.keys())]) 
vid_feat_inp = np.array([Y[key] for key in sorted(Y.keys())])

key_user = np.asarray(range(user_feat_inp.shape[0])).reshape(user_feat_inp.shape[0] ,1)
key_vid = np.asarray(range(vid_feat_inp.shape[0])).reshape(vid_feat_inp.shape[0] ,1)
user_feat_inp_w_key = np.concatenate((user_feat_inp ,key_user),axis = 1)
vid_feat_inp_w_key = np.concatenate((vid_feat_inp ,key_vid),axis = 1)
user_vid_time = trData.keys()

user_feat_inp_w_key_df = pd.DataFrame(user_feat_inp_w_key) 
vid_feat_inp_w_key_df = pd.DataFrame(vid_feat_inp_w_key)
user_vid_time_df = pd.DataFrame(user_vid_time)

rr = user_vid_time_df.sort_values([0 ,2]).reset_index()

h = pd.get_dummies(rr[1],prefix = 'vid_')
y_tr_p_w = pd.concat([rr.reset_index(drop = True), h], axis = 1)
y_tr_p_w.rename(columns = {0: 'user'}, inplace = True)
y_tr_p_w.rename(columns = {1: 'mov'}, inplace = True)
y_tr_p_w.rename(columns = {2: 'tim'}, inplace = True)
y_tr_p_w['desired'] = np.argmax(np.array(y_tr_p_w.iloc[:,4:]) ,1)

max_watch = (y_tr_p_w.groupby('user',axis = 0).sum().iloc[:
                                ,np.r_[4:(len(y_tr_p_w.columns)-2)]].sum(axis=1)).reset_index()
not_to_sample = (y_tr_p_w.groupby('user',axis = 0).sum().reset_index()).iloc[:
                                 ,np.r_[0 ,4:(len(y_tr_p_w.columns)-1)]]
to_sample = (not_to_sample.iloc[:,1:].apply(lambda y:np.where(y == 0),axis = 1)).reset_index(name = 'to_samp')
popularity = ((y_tr_p_w).sum(axis = 0)).iloc[4:-1]


vid_feat_inp_w_key_df.rename(columns = {300: 'mov_id'}, inplace = True)
user_feat_inp_w_key_df.rename(columns = {310: 'user_id'}, inplace = True)
user_vid_time_df.rename(columns = {0: 'user_id'}, inplace = True)
user_vid_time_df.rename(columns = {1: 'mov_id'}, inplace = True)
#user_w_vid_tim_and_feat = user_vid_time_df.merge(user_feat_inp_w_key_df 
#                                                 ,how = 'inner',on = 'user_id', sort = False)
user_vid_time_vidfeat = user_vid_time_df.merge(vid_feat_inp_w_key_df 
                                                             ,how = 'inner' ,on='mov_id' ,sort = False)
user_vid_time_vidfeat.rename(columns={'2_x': 'time_watch'} ,inplace=True)
user_vid_time_vidfeat_sorted = user_vid_time_vidfeat.sort_values(['user_id', 'time_watch']) 
cols_to_del = [user_vid_time_vidfeat_sorted.columns.get_loc("user_id")
             ,user_vid_time_vidfeat_sorted.columns.get_loc("mov_id")
             ,user_vid_time_vidfeat_sorted.columns.get_loc("time_watch")]

# ### Split training and testing data
tr_users = viewers * 0.8
tr_y = y_tr_p_w[y_tr_p_w['user'] < tr_users]
te_y = y_tr_p_w[~y_tr_p_w['user'].isin(tr_y['user'])]

In [104]:
reward_gamma = 0.9

In [130]:
tf.reset_default_graph()

In [131]:
with tf.name_scope('input'):
    inp = tf.placeholder("float32" ,[batch_size ,None, num_video] ,name = 'inp') 
    disc_pred_gen_ph = tf.placeholder("float32" ,[None, 1] ,name = 'inp_ph') 
    dynam_input = tf.placeholder("float32" ,[batch_size ,None ,n_feature] ,name = 'dynam_input') 
    const_input = tf.placeholder("float32" ,[batch_size ,n_user_feature] ,name = 'const_input') 
    y_true = tf.placeholder("int32",[None] ,name = 'Input_y')
    max_batch_length = tf.placeholder("float32" ,[batch_size] ,name = 'max_leng')
    r_c_idx = tf.placeholder("int32" ,[None,2] ,name = 'r_c_idx')
    r_c_n_idx = tf.placeholder("int32" ,[None,2] ,name = 'r_c_n_idx')
    tr_rw = tf.placeholder("float32" ,[None] ,name = 'rw_rep')
    _maxx = tf.placeholder("int32" ,name = 'maxx')
    block_mat = tf.placeholder("float32",[None, None] ,name = 'block')

In [132]:
with tf.variable_scope("weights"):
    W_gener_co = tf.get_variable("variable1", shape=[n_user_feature ,num_video]
                                        , initializer=tf.random_normal_initializer(), dtype='float32')
    W_gener_ou = tf.get_variable("variable2" ,shape=([n_hidden ,num_video])
                                        , initializer=tf.random_normal_initializer(), dtype='float32')
    tf.get_variable_scope().reuse_variables()
    assert tf.get_variable_scope().reuse==True

    W_gener_co_2 = tf.get_variable("variable1",shape=[n_user_feature, num_video]
                                          , dtype='float32')
    W_gener_ou_2 = tf.get_variable("variable2", shape=[n_hidden, num_video]
                                   , dtype='float32')

with tf.variable_scope("biases"):
    b_gener_co = tf.get_variable("variable1", shape=[num_video]
                                        , initializer=tf.random_normal_initializer(), dtype='float32')
    b_gener_ou = tf.get_variable("variable2", shape=[num_video]
                                        , initializer=tf.random_normal_initializer(), dtype='float32')

    tf.get_variable_scope().reuse_variables()
    assert tf.get_variable_scope().reuse==True
    b_gener_co_2 = tf.get_variable("variable1",shape=[num_video]
                                          , dtype='float32')
    b_gener_ou_2 = tf.get_variable("variable2",shape=[num_video]
                                          , dtype='float32')

with tf.variable_scope("biases_disc"):
    b_disc_co = tf.get_variable("variable1", shape=[n_user_feature]
                                        , initializer=tf.random_normal_initializer(), dtype='float32',)
    b_disc_ou_1 = tf.get_variable("variable2", shape=[n_user_feature]
                                        , initializer=tf.random_normal_initializer(), dtype='float32')
    b_disc_ou_2 = tf.get_variable("variable3", shape=[1]
                                        , initializer=tf.random_normal_initializer(), dtype='float32')

    tf.get_variable_scope().reuse_variables()
    assert tf.get_variable_scope().reuse==True
    b_disc_co_2 = tf.get_variable("variable1",shape=[n_user_feature]
                                          , dtype='float32')

    b_disc_ou_1_2 = tf.get_variable("variable2",shape=[n_user_feature]
                                          , dtype='float32')
    b_disc_ou_2_2 = tf.get_variable("variable3", shape=[1]
                                        , initializer=tf.random_normal_initializer(), dtype='float32')
    print(b_disc_co_2==b_disc_co)

with tf.variable_scope("weights_disc"):
    W_disc_co = tf.get_variable("variable1", shape=[n_user_feature ,n_user_feature]
                                        , initializer=tf.random_normal_initializer(), dtype='float32')
    W_disc_ou = tf.get_variable("variable2" ,shape=([n_hidden ,n_user_feature])
                                        , initializer=tf.random_normal_initializer(), dtype='float32')
    W_disc_ou_2 = tf.get_variable("variable3" ,shape=([n_user_feature ,1])
                                        , initializer=tf.random_normal_initializer(), dtype='float32')
    tf.get_variable_scope().reuse_variables()
    assert tf.get_variable_scope().reuse==True

    W_disc_co_2 = tf.get_variable("variable1",shape=[n_user_feature ,n_user_feature]
                                          , dtype='float32')
    W_disc_ou_1_2 = tf.get_variable("variable2", shape=[n_hidden ,n_user_feature]
                                   , dtype='float32')
    W_disc_ou_2_2 = tf.get_variable("variable3", shape=[n_user_feature ,1]
                                   , dtype='float32')
    print(W_disc_ou==W_disc_ou_1_2)


True
True


In [133]:
def pre_train_gener(dynam_input, const_input, max_batch_length
        ,W_gener_co ,W_gener_ou ,b_gener_co, b_gener_ou ,model, r_c_idx, r_c_n_idx):
    with tf.variable_scope('bb') as scope:
        lstm_cell = rnn_model(model ,n_hidden ,layers)
        outputs ,states = tf.nn.dynamic_rnn(lstm_cell ,inputs = dynam_input
                             ,dtype = tf.float32 ,sequence_length = max_batch_length)

    out_shaped = tf.reshape(outputs ,[-1 ,n_hidden])
    lay_2 = tf.reshape(tf.reshape(tf.matmul(out_shaped ,W_gener_ou) + b_gener_ou
               ,[-1,batch_size,num_video]) + (tf.matmul(const_input,
                                                        W_gener_co) + b_gener_co), [-1,num_video]) 
    
    p_w_d = tf.gather_nd(params = lay_2 ,indices = r_c_idx)
    p_w_neg = tf.gather_nd(params = lay_2 ,indices = r_c_n_idx)    
    return (lay_2 ,p_w_d ,p_w_neg) 

In [134]:
def reinforce_disc(inp, const_input, max_batch_length, W_disc_co, W_disc_ou, W_disc_ou_2, b_disc_co
                   , b_disc_ou_1, b_disc_ou_2, model):
    try:
        with tf.variable_scope('aa') as scope:
            lstm_cell = rnn_model(model ,n_hidden ,layers)
            outputs ,states = tf.nn.dynamic_rnn(lstm_cell ,inputs = inp
                                 ,dtype = tf.float32 ,sequence_length = max_batch_length)
    except:
        with tf.variable_scope('dd') as scope:
            lstm_cell = rnn_model(model ,n_hidden ,layers)
            outputs ,states = tf.nn.dynamic_rnn(lstm_cell ,inputs = inp
                                 ,dtype = tf.float32 ,sequence_length = max_batch_length)
   
    out_shaped = tf.reshape(outputs ,[-1 ,n_hidden])
    print (out_shaped.get_shape())
    
    lay_2 = tf.reshape(tf.reshape(tf.matmul(out_shaped ,W_disc_ou) + b_disc_ou_1, [-1, batch_size, n_user_feature]) 
                       + (tf.matmul(const_input, W_disc_co) + b_disc_co), [-1,n_user_feature])
    
    print(lay_2.get_shape())
    
    lay_3 = tf.matmul(lay_2 ,W_disc_ou_2) + b_disc_ou_2
    print(lay_3.get_shape())
    
    return (lay_3) 

In [135]:
def reinforce_gener(inp, const_input, max_batch_length,
        W_gener_co_2, W_gener_ou_2, b_gener_co_2, b_gener_ou_2, model, _maxx):
    lstm_cell = rnn_model(model ,n_hidden ,layers)

    outputs ,states = tf.nn.dynamic_rnn(lstm_cell ,inputs = dynam_input
                             ,dtype = tf.float32 ,sequence_length = max_batch_length)

    with tf.variable_scope('cc') as scope:
        lstm_cell = rnn_model(model ,n_hidden ,layers)
        outputs ,states = tf.nn.dynamic_rnn(lstm_cell ,inputs = dynam_input
                             ,dtype = tf.float32 ,sequence_length = max_batch_length)

   
    out_shaped = tf.reshape(outputs ,[-1 ,n_hidden])
    lay_2 = tf.reshape(tf.reshape(tf.matmul(out_shaped ,W_gener_ou_2) + b_gener_ou_2
               ,[-1,batch_size,num_video]) + (tf.matmul(const_input,
                                                        W_gener_co_2) + b_gener_co_2), [-1,num_video]) 
    
   
    lay_3 = tf.nn.softmax(lay_2)
    lay_4 = tf.reduce_max(lay_3, 1)
    return (lay_3, lay_4)  

In [136]:
y_pred_gener, y_pred_gen_max = reinforce_gener(inp, const_input, max_batch_length,
        W_gener_co_2, W_gener_ou_2, b_gener_co_2, b_gener_ou_2, model, _maxx)

In [137]:
y_pred, p_w_d, p_w_neg = pre_train_gener(dynam_input, const_input 
        ,max_batch_length
        ,W_gener_co, W_gener_ou, b_gener_co, b_gener_ou, model, r_c_idx, r_c_n_idx)

In [138]:
 disc_pred_gen = reinforce_disc(inp, const_input, max_batch_length, W_disc_co, W_disc_ou, W_disc_ou_2, b_disc_co
                   , b_disc_ou_1, b_disc_ou_2, model)  

(?, 32)
(?, 310)
(?, 1)


In [139]:
disc_pred_real = reinforce_disc(inp, const_input, max_batch_length, W_disc_co_2, W_disc_ou_1_2, W_disc_ou_2_2
                                , b_disc_co_2, b_disc_ou_1_2, b_disc_ou_2_2, model)

(?, 32)
(?, 310)
(?, 1)


In [140]:
decays = tf.tile(tf.exp(tf.log(reward_gamma) * tf.to_float(tf.range((_maxx)))), [batch_size])
prob_disc_gen = tf.sigmoid(disc_pred_gen_ph)
rewards = tf.matmul(block_mat, tf.reshape(tf.reshape(tf.reshape(decays,[-1,1]) * prob_disc_gen, [-1]), [-1, 1]))

In [141]:
with tf.name_scope('pair_wise'):
    loss_p_w = -tf.reduce_mean(tf.multiply(tf.log(tf.sigmoid(tf.subtract(p_w_d ,p_w_neg))) ,tr_rw))
    regularizer = tf.nn.l2_loss(tf.abs(W_gener_ou_2)) 
    p_wise = tf.reduce_mean(loss_p_w + beta * regularizer)

with tf.name_scope('loss_gen'):
    g_loss = -tf.reduce_mean(tf.log(y_pred_gen_max) * rewards)

with tf.name_scope('loss_d_gen'):
       loss_d_gen = tf.reduce_mean(tf.multiply(
         tf.nn.sigmoid_cross_entropy_with_logits(
             logits = disc_pred_gen, labels = tf.zeros([tf.shape(disc_pred_gen)[0], 1])), tr_rw))

with tf.name_scope('loss_d_real'):
       loss_d_real = tf.reduce_mean(tf.multiply(
         tf.nn.sigmoid_cross_entropy_with_logits(
             logits = disc_pred_real, labels = tf.ones([tf.shape(disc_pred_real)[0], 1])), tr_rw))

with tf.name_scope('train_d_real'):    
    optimizer_d_r = tf.train.GradientDescentOptimizer(learning_rate = lr_rat).minimize(loss_d_real)

with tf.name_scope('train_d_gen'):    
    optimizer_d_gen = tf.train.GradientDescentOptimizer(learning_rate = lr_rat).minimize(loss_d_gen) 

with tf.name_scope('train_gen'):    
    optimizer_gen = tf.train.GradientDescentOptimizer(learning_rate = lr_rat).minimize(g_loss)
    
with tf.name_scope('pre_train_gen'):    
    optimizer_gen_pre = tf.train.AdamOptimizer(learning_rate = lr_rat).minimize(p_wise)   
    

In [142]:
init = tf.global_variables_initializer()
sess = tf.Session() 
sess.run(init)

In [143]:
g_steps = 1
d_steps = 1
proportion_supervised = 0.5
proportion_generated = 0.5

In [146]:
for epoch in range(int(n_opt_epoch)):
    for batch_n in range(int(n_users / batch_size)):
        rw_to_chose = np.random.choice(int(tr_users) ,128 ,replace = False)
        const_tr = (sort_pd_df_by_ext_vec(user_feat_inp_w_key_df, rw_to_chose,
                                          cols = ['user_id'])).drop(['user_id'], axis = 1)
        x_tr = sort_pd_df_by_ext_vec(user_vid_time_vidfeat_sorted,
                                      rw_to_chose, cols = ['user_id','time_watch'])
        d_real_inp = sort_pd_df_by_ext_vec(y_tr_p_w, rw_to_chose, cols = ['user','tim'])
        y_batch = sort_pd_df_by_ext_vec(tr_y ,rw_to_chose,cols = ['user','tim'])
        y_batch_p_w_idx_arr = np.array(y_batch['desired'].reset_index())
        to_sample_sess = sort_pd_df_by_ext_vec(df = to_sample,ext_sor_vec = rw_to_chose,cols = ['index'])
        length_vec = sort_pd_df_by_ext_vec(df = max_watch ,ext_sor_vec = rw_to_chose ,cols = ['user'])      
        length_max = max(length_vec.iloc[:,1])
        str_idx = algeb_geom_series(0 ,start = 0 ,jump = length_max ,length = batch_size)
        end_idx = np.append(length_max ,length_vec.iloc[1:,1:] + str_idx[1:].reshape(batch_size-1,-1))
        app_range = range_bet_col_t_col_n_append(str_idx, end_idx)
        ind_r_t_pick = (np.isin(range(int(length_max * batch_size)) ,app_range)).astype(int)
        rep_ind = np.repeat(ind_r_t_pick,10)
        trial_size = (batch_size * length_max, n_feature)
        trial_size_d = (batch_size * length_max, num_video)
        zero_array_x = np.zeros(trial_size); zero_array_y = np.zeros(batch_size * length_max); zero_array_d_r =np.zeros(trial_size_d) 
        x_tr, y_tr = np_pad_tr_x_n_tr_y(x_tr.iloc[:,3:], y_batch, batch_size, str_idx.astype(int),
                                        zero_array_x, zero_array_y, length_vec.iloc[:,1].astype(int))
        d_real_inp = np_pad_tr_x(d_real_inp.iloc[:,4:-1], batch_size, str_idx.astype(int),
                                        zero_array_d_r, length_vec.iloc[:,1].astype(int))
        neg_sample, idx_p_w, rep_seq = negtv_sample(to_sample_sess ,popularity
                                                    ,batch_size ,length_max ,num_to_sample = 10)
        idx = np.repeat(y_tr,10)       
        idx_pp = np.concatenate((rep_seq ,idx.reshape(-1,1)),axis = 1)
        block_tri = block_tri_mat(length_max, batch_size)
        
        
        if (epoch < 1): # pre_train 
            _, g_p_w, g_pred = sess.run([optimizer_gen_pre, p_wise, y_pred], feed_dict = 
                                         {dynam_input: x_tr.reshape(batch_size,-1,n_feature)
                                                  ,const_input: const_tr 
                                                  ,tr_rw: rep_ind
                                                  ,r_c_idx: idx_pp
                                                  ,r_c_n_idx: idx_p_w
                                                  ,max_batch_length: length_vec.iloc[:,1]})
        else:
            for _ in range(g_steps):
                if np.random.random() < proportion_supervised:
                    _, _g_p_w, _g_pred = sess.run([optimizer_gen_pre, p_wise, y_pred], feed_dict = 
                                         {dynam_input: x_tr.reshape(batch_size,-1,n_feature)
                                                  ,const_input: const_tr 
                                                  ,tr_rw: rep_ind
                                                  ,r_c_idx: idx_pp
                                                  ,r_c_n_idx: idx_p_w
                                                  ,y_true: y_tr
                                                  ,max_batch_length: length_vec.iloc[:,1]})
                else: 
                    gen_xx = sess.run(y_pred_gener, feed_dict = {
                            dynam_input: x_tr.reshape(batch_size, -1, n_feature)
                                                  ,const_input: const_tr 
                                                  ,max_batch_length: length_vec.iloc[:,1]
                                                })
                    
                    d_xx = sess.run(disc_pred_gen, feed_dict = {inp: gen_xx.reshape(batch_size, -1, num_video)
                            ,const_input: const_tr
                            ,max_batch_length: length_vec.iloc[:,1]})
                    
                    
                    _, g_l, unsupervised_gen_x, _rewards = sess.run([optimizer_gen, g_loss, y_pred_gener, rewards]
                                                                    , feed_dict = 
                                         {dynam_input: x_tr.reshape(batch_size, -1, n_feature),
                                                  _maxx: length_max, 
                                                  disc_pred_gen_ph: d_xx,
                                                  block_mat: block_tri 
                                                  ,const_input: const_tr 
                                                  ,max_batch_length: length_vec.iloc[:, 1]})
        if ((epoch >= 1) & (epoch <= 2)) :
             _, _disc_pred_real_pre, d_loss_pre = sess.run([optimizer_d_r, disc_pred_real, loss_d_real], feed_dict = 
                                                         {inp: d_real_inp.reshape(batch_size, -1, num_video)
                                                          ,const_input: const_tr
                                                          ,max_batch_length: length_vec.iloc[:,1]
                                                          ,tr_rw: rep_ind 
                            
                        })
            
        if (epoch > 2):
            for _ in range(d_steps):
                if np.random.random() < proportion_generated:
                    _gen_xx = sess.run(y_pred_gener, feed_dict = {
                            dynam_input: x_tr.reshape(batch_size, -1, n_feature)
                                                  ,const_input: const_tr 
                                                  ,max_batch_length: length_vec.iloc[:,1]
                                    })
                    _, _disc_pred_gen, d_loss = sess.run([optimizer_d_gen, disc_pred_gen, loss_d_gen], feed_dict = 
                                                   {inp: _gen_xx.reshape(batch_size, -1, num_video)
                                                    ,const_input: const_tr
                                                    ,max_batch_length: length_vec.iloc[:,1]})
                else:
                    _, _disc_pred_real, _d_loss = sess.run([optimizer_d_r, disc_pred_real, loss_d_real], feed_dict = 
                                                         {inp: d_real_inp.reshape(batch_size, -1, num_video)
                                                          ,const_input: const_tr
                                                          ,max_batch_length: length_vec.iloc[:,1]
                                                          ,tr_rw: rep_ind 
                            
                        })


/Users/onivron/anaconda/envs/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/onivron/anaconda/envs/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/onivron/anaconda/envs/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
